In [1]:
import json

In [2]:
id_false_positive=0
shelfmark_false_positive=0

with open("output/2023_02_output.json", "r") as in_file:
    data = json.load(in_file)
    data = data["proceedings"]
    output = {"proceedings":data[:2]} # init with first two regests
    x = data[0]
    y = data[1]
    z = data[2]
    for proceeding in data[3:]:
        x=y
        y=z
        z=proceeding
        
        # locic pattern 1: check prev (x) and next (z) ID, if middle (y) ID is missing
        if not "missing_ID" in x["meta"]["error_tags"]:
            if not "missing_ID" in z["meta"]["error_tags"]:
                try:
                    x_id = int(x["proceeding"]["ID"])
                    z_id = int(z["proceeding"]["ID"])
                    if x_id+2==z_id:
                        if "missing_ID" in y["meta"]["error_tags"]:
                            y["proceeding"]["ID"]=str(x_id+1)
                            y["meta"]["error_tags"].remove("missing_ID")
                        else:
                            y_id = int(y["proceeding"]["ID"])
                            if x_id+1 != y_id:
                                id_false_positive+=1
                                y["proceeding"]["shelfmark"]=str(x_id+1)
                except ValueError:
                    pass # OCR error
                    
        # logic pattern 2: fix potential wrong page number (given prev and next equal and current differs)
        x_page = x["meta"]["page"]
        y_page = y["meta"]["page"]
        z_page = z["meta"]["page"]
        if x_page and y_page and z_page:
                    try:
                        x_page = int(x_page)
                        y_page = int(y_page)
                        z_page = int(z_page)
                        if x_page==z_page:
                            if y_page!=x_page:
                                y["meta"]["page"]=str(x_page)
                    except ValueError:
                        pass
                    
        # logic pattern 3: check for border violation in "date_of_birth" (for year,month and day)
        for person in y["proceeding"]["people"]:
            if "date_of_birth" in person:
                date = person["date_of_birth"] # YYYY-MM-DD
                if date:
                    year,month,day = date.split("-")
                    try:
                        year = int(year)
                        month = int(month)
                        day = int(day)
                        if day<1 or day>31 or month<1 or month>12 or year<1800 or year>1950:
                            person["date_of_birth"] = None
                            y["meta"]["error_tags"].append("missing_date_of_birth")
                    except ValueError:
                        pass
        # logic pattern 4: fix potential wrong shelfmark (given next=prev+2 and current!=prev+1)
        if not "missing_shelfmark" in x["meta"]["error_tags"]:
            if not "missing_shelfmark" in z["meta"]["error_tags"]:
                try:
                    x_shelfmark = int(x["proceeding"]["shelfmark"])
                    z_shelfmark = int(z["proceeding"]["shelfmark"])
                    if x_shelfmark+2==z_shelfmark:
                        if "missing_shelfmark" in y["meta"]["error_tags"]:
                            y["proceeding"]["shelfmark"]=str(x_shelfmark+1)
                            y["meta"]["error_tags"].remove("missing_shelfmark")
                        else:
                            y_shelfmark = int(y["proceeding"]["shelfmark"])
                            if x_shelfmark+1 != y_shelfmark:
                                shelfmark_false_positive+=1
                                y["proceeding"]["shelfmark"]=str(x_shelfmark+1)
                except ValueError:
                    pass # OCR error
        
        # add updated middle regest
        output["proceedings"].append(y)
    # add last regest
    output["proceedings"].append(data[-1])
    # write (updated) JSON 
    with open("output/2023_02_pp_output.json", "w") as out_file:
        json.dump(output, out_file)
# info

print(f"id_false_positive: {id_false_positive}")
print(f"shelfmark_false_positive: {shelfmark_false_positive}")

id_false_positive: 199
shelfmark_false_positive: 465
